In this example, imagine you have a list of order IDs. For each order, the system needs to:

 - Fetch product details from a database (simulated with sleep).
 - Check the inventory status from an external API (simulated with sleep).
 - Calculate the final price.

Below is the solution to this problem - the Synchronous way

In [1]:
import time

def fetch_product_data(order_id):
    print(f"Fetching data for Order {order_id}...")
    time.sleep(1)  # Simulating network latency
    return {"id": order_id, "price": 100}

def check_inventory(order_id):
    print(f"Checking inventory for Order {order_id}...")
    time.sleep(1)  # Simulating API call
    return True

def process_order(order_id):
    product = fetch_product_data(order_id)
    in_stock = check_inventory(order_id)
    
    if in_stock:
        final_price = product["price"] * 1.05
        print(f"Order {order_id} processed. Final Price: ${final_price}")
    else:
        print(f"Order {order_id} out of stock.")

def main():
    orders = [101, 102, 103]
    start_time = time.perf_counter()

    for order_id in orders:
        process_order(order_id)

    end_time = time.perf_counter()
    print(f"--- All orders processed in {end_time - start_time:.2f} seconds ---")

if __name__ == "__main__":
    main()

Fetching data for Order 101...
Checking inventory for Order 101...
Order 101 processed. Final Price: $105.0
Fetching data for Order 102...
Checking inventory for Order 102...
Order 102 processed. Final Price: $105.0
Fetching data for Order 103...
Checking inventory for Order 103...
Order 103 processed. Final Price: $105.0
--- All orders processed in 6.05 seconds ---


Below we have converted the whole syncronous program into asynchronous.
The total time is now drastically reduced to just 1 sec.

We are following this Golden Rule: If you want things to happen at the same time, you must either await a non-blocking call or offload a blocking call to a thread.

Here the blocking calls are time.sleep(1) which simulates API call/ NW latency -> we have offloaded them from main thread to a side thread that is asynchronous.

In [8]:
import time
import asyncio

def blocking_call(t):
    time.sleep(t)

async def fetch_product_data(order_id):
    print(f"Fetching data for Order {order_id}...")
    await asyncio.to_thread(blocking_call, 1)  # Simulating network latency
    return {"id": order_id, "price": 100}

async def check_inventory(order_id):
    print(f"Checking inventory for Order {order_id}...")
    await asyncio.to_thread(blocking_call, 1)  # Simulating API call
    return True

async def process_order(order_id):
    product, in_stock = await asyncio.gather(fetch_product_data(order_id), check_inventory(order_id))
    
    if in_stock:
        final_price = product["price"] * 1.05
        print(f"Order {order_id} processed. Final Price: ${final_price}")
    else:
        print(f"Order {order_id} out of stock.")

async def main():
    orders = [101, 102, 103]
    start_time = time.perf_counter()

    tasks = [asyncio.create_task(process_order(order_id)) for order_id in orders]
    await asyncio.gather(*tasks)

    end_time = time.perf_counter()
    print(f"--- All orders processed in {end_time - start_time:.2f} seconds ---")

if __name__ == "__main__":
    await main()

Fetching data for Order 101...
Checking inventory for Order 101...
Fetching data for Order 102...
Checking inventory for Order 102...
Fetching data for Order 103...
Checking inventory for Order 103...
Order 103 processed. Final Price: $105.0
Order 101 processed. Final Price: $105.0
Order 102 processed. Final Price: $105.0
--- All orders processed in 1.01 seconds ---


There is still a major problem that can occur in real-world production systems -> what if there are 1000 orders to process?

With our current solution this will allow Python to open as many threads as required. This may exhaust our system's memory or crash DB.

We need to limit the threads. We use Semaphore for this.
Think of it like a "bouncer" at a club—it only lets a specific number of tasks perform their "heavy" work at the same time.


In [13]:
import time
import asyncio

def blocking_call(t):
    time.sleep(t)

thread_limit = asyncio.Semaphore(3)     # limiting asyncio threads to 3 at a time

async def fetch_product_data(order_id):
    async with thread_limit:
        print(f"Fetching data for Order {order_id}...", flush=True)
        await asyncio.to_thread(blocking_call, 1)  # Simulating network latency
        return {"id": order_id, "price": 100}

async def check_inventory(order_id):
    async with thread_limit:
        print(f"Checking inventory for Order {order_id}...", flush=True)
        await asyncio.to_thread(blocking_call, 1)  # Simulating API call
        return True

async def process_order(order_id):
    product, in_stock = await asyncio.gather(fetch_product_data(order_id), check_inventory(order_id))
    
    if in_stock:
        final_price = product["price"] * 1.05
        print(f"Order {order_id} processed. Final Price: ${final_price}")
    else:
        print(f"Order {order_id} out of stock.")

async def main():
    orders = [101, 102, 103, 105, 106, 110]
    start_time = time.perf_counter()

    tasks = [asyncio.create_task(process_order(order_id)) for order_id in orders]
    await asyncio.gather(*tasks)

    end_time = time.perf_counter()
    print(f"--- All orders processed in {end_time - start_time:.2f} seconds ---")

if __name__ == "__main__":
    await main()

Fetching data for Order 101...
Checking inventory for Order 101...
Fetching data for Order 102...
Checking inventory for Order 102...
Fetching data for Order 103...
Checking inventory for Order 103...
Order 101 processed. Final Price: $105.0
Fetching data for Order 105...
Order 102 processed. Final Price: $105.0
Checking inventory for Order 105...
Fetching data for Order 106...
Order 103 processed. Final Price: $105.0
Checking inventory for Order 106...
Fetching data for Order 110...
Checking inventory for Order 110...
Order 105 processed. Final Price: $105.0
Order 106 processed. Final Price: $105.0
Order 110 processed. Final Price: $105.0
--- All orders processed in 4.05 seconds ---


Now that we handled the max number of threads Python can allocate, there is one more concern - What if any thread hangs forever (e.g. a database query or an API call). 

To handle this scenario, we need to have Timeout defined in our program.

This is done below :


In [25]:
import time
import asyncio

def blocking_call(t):
    time.sleep(t)

thread_limit = asyncio.Semaphore(3)     # limiting asyncio threads to 3 at a time

async def fetch_product_data(order_id) -> dict:
    async with thread_limit:
        print(f"Fetching data for Order {order_id}...", flush=True)
        await asyncio.to_thread(blocking_call, 1)  # Simulating network latency
        return {"id": order_id, "price": 100}

async def check_inventory(order_id) -> bool:
    async with thread_limit:
        print(f"Checking inventory for Order {order_id}...", flush=True)
        await asyncio.to_thread(blocking_call, 1)  # Simulating API call
        return True

async def process_order(order_id, timeout):
    try:
        product = await asyncio.wait_for(fetch_product_data(order_id), timeout=timeout)
        in_stock = await asyncio.wait_for(check_inventory(order_id), timeout=timeout)
        
        if in_stock:
            final_price = product["price"] * 1.05
            print(f"Order {order_id} processed. Final Price: ${final_price}")
        else:
            print(f"Order {order_id} out of stock.")
    except TimeoutError:
        print(f"FAILED: TimeoutError - Order {order_id} can not be processed")

async def main():
    orders = [101, 102, 103]
    start_time = time.perf_counter()

    tasks = [
        asyncio.create_task(process_order(order_id, timeout=3)) 
        for order_id in orders
        ]
    await asyncio.gather(*tasks)

    end_time = time.perf_counter()
    print(f"--- All orders processed in {end_time - start_time:.2f} seconds ---")

if __name__ == "__main__":
    await main()

Fetching data for Order 101...
Fetching data for Order 102...
Fetching data for Order 103...
Checking inventory for Order 103...
Checking inventory for Order 101...
Checking inventory for Order 102...
Order 102 processed. Final Price: $105.0
Order 101 processed. Final Price: $105.0
Order 103 processed. Final Price: $105.0
--- All orders processed in 2.01 seconds ---


With this we've built s program that handles blocking code, runs tasks in parallel, avoids race conditions, limits concurrency, and handles individual failures/timeouts without crashing the whole script.